## Project: AI-BEMS 알고리즘 도출
* 2024.05.21. SQI에서 인천광역시 고등학교 17개소의 BEMS 데이터 공유

In [80]:
import pandas as pd
import numpy as np
import os

file_path = '/home/ymatics/CodingSpace/AI_BEMS/data_colec_h_202405211344_15_2GB.csv'
output_dir = '/home/ymatics/CodingSpace/AI_BEMS/building_data'  # 출력 파일을 저장할 디렉토리

In [ ]:
import pandas as pd
import numpy as np
import os

file_path = '/home/ymatics/CodingSpace/AI_BEMS/data_colec_h_202405211344_15_2GB.csv'
output_dir = '/home/ymatics/CodingSpace/AI_BEMS/building_data'  # 출력 파일을 저장할 디렉토리

In [2]:
import pandas as pd
import os

file_path = '/home/ymatics/CodingSpace/AI_BEMS/building_data/buildingID_B0008.csv'  # 용학초등학교 'B0008'
df_buildID = pd.read_csv(file_path)

df_buildID['colec_dt'] = pd.to_datetime(df_buildID['colec_dt']).dt.floor('min')  # 분 이하는 제거

dev_id_list = df_buildID['dev_id'].unique()
print(f"{len(dev_id_list)=}\n{dev_id_list=}")
Inverter = np.sum((dev_id_list >= 1000) & (dev_id_list < 2000))
ElecMeter = np.sum((dev_id_list >= 2000) & (dev_id_list < 3000))
GasMeter = np.sum((dev_id_list >= 3000) & (dev_id_list < 4000))
HeatMeter = np.sum((dev_id_list >= 4000) & (dev_id_list < 5000))
EnvSensor = np.sum((dev_id_list >= 5000) & (dev_id_list < 6000))
AirCon = np.sum((dev_id_list >= 6000) & (dev_id_list < 7000))
print(f'{Inverter=}, {ElecMeter=}, {GasMeter=}, {HeatMeter=}, {EnvSensor=}, {AirCon=}')

len(dev_id_list)=133
dev_id_list=array([1001, 1002, 2001, 2002, 2003, 2004, 3001, 3002, 3003, 3004, 3005,
       4001, 4002, 5001, 5002, 5003, 6001, 6002, 6003, 6004, 6005, 6006,
       6007, 6008, 6009, 6010, 6011, 6012, 6013, 6014, 6015, 6016, 6017,
       6018, 6019, 6020, 6021, 6022, 6023, 6024, 6025, 6026, 6027, 6028,
       6029, 6030, 6031, 6032, 6033, 6034, 6035, 6036, 6037, 6038, 6039,
       6040, 6041, 6042, 6043, 6044, 6045, 6046, 6047, 6048, 6049, 6050,
       6051, 6052, 6053, 6054, 6055, 6056, 6057, 6058, 6059, 6060, 6061,
       6062, 6063, 6064, 6065, 6066, 6067, 6068, 6069, 6070, 6071, 6072,
       6073, 6074, 6075, 6076, 6077, 6078, 6079, 6080, 6081, 6082, 6083,
       6084, 6085, 6086, 6087, 6088, 6089, 6090, 6091, 6092, 6093, 6094,
       6095, 6096, 6097, 6098, 6099, 6100, 6101, 6102, 6103, 6104, 6105,
       6106, 6107, 6108, 6109, 6110, 6111, 6112, 6113, 6114, 6115, 6116,
       6117])
Inverter=2, ElecMeter=4, GasMeter=5, HeatMeter=2, EnvSensor=3, AirCon=117


In [30]:
def print_tagCD(df, devID):
    df_devID = df[df['dev_id'] == devID]
    print('tagCD list: ', df_devID['tag_cd'].unique())


def select_devID_tagCD(df, devID, tagCD):
    return df[(df['dev_id'] == devID) & (df['tag_cd'] == tagCD)]


#ToDo: diff의 NaN 값 처리 필요
def calc_tagCD_diff(df_devID_tagCD):
    df_diff = df_devID_tagCD.copy()
    df_diff['colec_val'] = pd.to_numeric(df_diff['colec_val'], errors='coerce')
    df_diff['colec_val'] = df_diff['colec_val'].diff().fillna(0)

    # 최대값 인덱스를 찾아 해당 값을 zero로 변경 --- 삭제
    max_index = df_diff['colec_val'].idxmax()
    df_diff.at[max_index, 'colec_val'] = 0
    
    return df_diff


def plot_devID_tagCD(dfL, legendL, device_name=None, fig=True, W=None, H=None):
    import plotly.graph_objects as go
    if fig:
        fig = go.Figure()
        for idx, df in enumerate(dfL):
            collect_dt = dfL[idx]['colec_dt']
            cal_max = df['colec_val'].max()
            normalized_val = df['colec_val'] / cal_max if abs(cal_max) > 0.01 else df['colec_val'] / (cal_max + 0.01)
            fig.add_trace(
                go.Scatter(x=collect_dt, y=normalized_val, 
                           mode='markers', marker=dict(size=4), name=legendL[idx]+f', max={cal_max:.1f}')
            )
        
        # 레이아웃 설정
        layout_params = {
            'title': dict(text=device_name, x=0.5, xanchor='center'),
            'yaxis': dict(range=[-0.5, 1.5]),
            'xaxis_title': 'Collect Date',
            'yaxis_title': 'Normalized Value',
            'xaxis': dict(
                tickmode='linear',  # x 축 기준선 매 시각마다 표시
                dtick=3600000  # 1시간 단위로 표시 (3600000 밀리초)
            ),
            'legend': dict(x=0.01, y=0.98, traceorder='normal', 
                           bgcolor='rgba(255, 255, 255, 0.5)', bordercolor='rgba(0, 0, 0, 0.5)', borderwidth=1)
        }
        
        # width와 height가 주어지면 추가
        if W is not None:
            layout_params['width'] = W
        if H is not None:
            layout_params['height'] = H
        
        fig.update_layout(**layout_params)
        
        # 모든 트레이스를 처음에 보이도록 설정
        fig.update_traces(visible=True)
        
        fig.show()
    else:
        pass

### EDA Date Period

In [7]:
# 우선 일주일치만 가지고 고찰
#df = df_buildID
#start_date, end_date = pd.to_datetime('2024-01-21 00:00:00'), pd.to_datetime('2024-01-28 00:00:00')  # 2024년 1월 21일(일)~27일(토)
start_date, end_date = pd.to_datetime('2024-01-22 00:00:00'), pd.to_datetime('2024-01-24 00:00:00')  # 2024년 1월 22일(월)~23일(화)
df = df_buildID[(df_buildID['colec_dt'] >= start_date) & (df_buildID['colec_dt'] <= end_date)]

### Device Analysis

In [8]:
devID, device_name = 1001, '인버터1'
tag_dict = {10002: 'PV전압', 20002: 'PV전류', 30001: '현재출력', 30002: 'PV출력', 60001: '주파수', 70006: '온도', 202202: 'selfcheck'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

plot_devID_tagCD(tag_data, tag_names, device_name, fig=False)
print_tagCD(df, devID)

tagCD list:  [  10002   10003   10004   10005   20002   20003   20004   20005  202202
   30001   30002   40001   60001     703 9003005 9003007 9003008 9003014
 9003020 9003023 9003030 9003034 9003038 9003040 9003041 9003042 9003045
 9003047 9003048 9003049 9003081 9003106]


In [9]:
devID, device_name = 2001, '학교MAIN'
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량_차', 50001: '역률', 202202: 'selfcheck'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
tags[40004] = calc_tagCD_diff(tags[40004])
tags[40007] = calc_tagCD_diff(tags[40007])

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

plot_devID_tagCD(tag_data, tag_names, device_name, fig=False)
print_tagCD(df, devID)

tagCD list:  [202202  30001  40004  40007  50001]


In [10]:
devID, device_name = 2002, 'EHP-1'
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량', 50001: '역률', 202202: 'selfcheck'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
tags[40004] = calc_tagCD_diff(tags[40004])

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

plot_devID_tagCD(tag_data, tag_names, device_name, fig=False)
print_tagCD(df, devID)

tagCD list:  [202202  30001  40004  50001]


In [12]:
device_dict = {
    2002: "EHP-1",
    2003: "EHP-2",
    2004: "EHP-4",
}

#tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량', 50001: '역률', 202202: 'selfcheck'}
tag_dict = {30001: '현재 출력'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="all EHPs", fig=False, W=None, H=600)

In [86]:
#ToDo. check

# device_name = '인천용학초등학교: 식당급탕보일러'
devID = 3001  # 3001 ~ 3005
print_tagCD(df, devID)
df_devID_tag0 = select_devID_tagCD(df, devID, tagCD=202202)  # selfcheck
df_devID_tagA = select_devID_tagCD(df, devID, tagCD=70002)  # 보정적산량
df_devID_tagA_diff = df_devID_tagA.copy()
df_devID_tagA_diff['colec_val'] = pd.to_numeric(df_devID_tagA_diff['colec_val'], errors='coerce')
df_devID_tagA_diff['colec_val'] = df_devID_tagA_diff['colec_val'].diff().fillna(0)
plot_devID_tagCD([df_devID_tag0, df_devID_tagA_diff], ['selfcheck', '보정적산량_diff'], device_name, fig=False)

tagCD list:  [202202  70002]


In [ ]:
# ToDo. 일단 아래 루틴 사용

devID, device_name = 2002, '본관급탕보일러'
tag_dict = {90001: '공급온도', 90002: '환수온도', 90003: '순시유량', 90004: '누적유량_차', 90005: '순시열량', 90006: '누적열량_차', 202202: 'selfcheck'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
tags[90004] = calc_tagCD_diff(tags[90004])
tags[90006] = calc_tagCD_diff(tags[90006])

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

plot_devID_tagCD(tag_data, tag_names, device_name, fig=False)
print_tagCD(df, devID)

In [85]:
device_name = '인천용학초등학교: 본관급탕보일러(열량계)'
devID = 4001  # 4001 ~ 4002
print_tagCD(df, devID)
df_devID_tag0 = select_devID_tagCD(df, devID, tagCD=202202)  # selfcheck
df_devID_tagA = select_devID_tagCD(df, devID, tagCD=90001)  # 공급온도
df_devID_tagB = select_devID_tagCD(df, devID, tagCD=90002)  # 환수온도
df_devID_tagC = select_devID_tagCD(df, devID, tagCD=90003)  # 순시유량
df_devID_tagD = select_devID_tagCD(df, devID, tagCD=90004)  # 누적유량
df_devID_tagE = select_devID_tagCD(df, devID, tagCD=90005)  # 순시열량
df_devID_tagF = select_devID_tagCD(df, devID, tagCD=90006)  # 누적열량
plot_devID_tagCD([df_devID_tag0, df_devID_tagA, df_devID_tagB, df_devID_tagC, df_devID_tagE], 
                 ['selfcheck', '공급온도', '환수온도', '순시유량', '순시열량'], device_name, fig=False)

tagCD list:  [202202  90001  90002  90003  90004  90005  90006]


In [13]:
devID, device_name = 5001, '외기온도'
tag_dict = {80002: '온도', 80003: '습도', 80004: 'CO2', 80005: '미세먼지', 80006: '초미세먼지', 80007: '조도', 202202: 'selfcheck'}
tag_dict = {80002: '온도', 80003: '습도', 80004: 'CO2', 80007: '조도', 202202: 'selfcheck'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[90004] = calc_tagCD_diff(tags[90004])

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

plot_devID_tagCD(tag_data, tag_names, device_name, fig=False)
print_tagCD(df, devID)

tagCD list:  [202202  80002  80003  80004  80005  80006  80007]


In [34]:
devID, device_name = 6001, '4-7반_본405'
tag_dict = {100001: '운전선택탭', 100002: '냉난방모드', 100003: '현재온도', 100004: '희망온도', 100005: '풍량', 100006: '리모컨허용', 100007: '에러', 202202: 'selfcheck'}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[90004] = calc_tagCD_diff(tags[90004])

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

plot_devID_tagCD(tag_data, tag_names, device_name, fig=False)
print_tagCD(df, devID)

tagCD list:  [100001 100002 100003 100004 100005 100006 100007 202202]


In [5]:
device_dict = {
    6001: "4-7반_본405",
    6002: "유치원복도1",
    6003: "유치원교사실_후108",
    6004: "누리반_후107",
    6005: "한울반_후106",
    6006: "가람반_후105",
    6007: "1-6반_본108",
    6008: "1-5반_본107",
    6009: "1-4반_본106",
    6010: "1-3반_본105",
    6011: "체육관후면좌_후210",
    6012: "체육관전면좌_후210",
    6013: "음악실1_본501-2",
    6014: "음악실1_본501-2",
    6015: "3-4반_후207",
    6016: "음악준비실_본501-4",
    6017: "3-3반_후206",
    6018: "개인연습실2_본501-5",
    6019: "3-2반_후205",
    6020: "개인연습실1_본501-6",
    6021: "3-1반_후204",
    6022: "무용실_본501",
    6023: "3학년연구실_후203",
    6024: "무용실_본501",
    6025: "컴준비실_후208",
    6026: "컴퓨터실_후208-1",
    6027: "컴퓨터실_후208-1",
    6028: "1-7반_본210",
    6029: "1-8반_본209",
    6030: "3-8반_본208",
    6031: "3-7반_본207",
    6032: "3-6반_본206",
    6033: "행정실2_본203",
    6034: "행정실1_본203",
    6035: "탕비실_본204-2",
    6036: "교무실2_본202",
    6037: "교무실1_본202",
    6038: "교장실_본201",
    6039: "학교운영위원회실_본212",
    6040: "3-5반_본205",
    6041: "돌봄협의실_후302",
    6042: "돌봄자료실_후307",
    6043: "방과후코디실_후307-1",
    6044: "2-2반_후307-1",
    6045: "2-5반_후304",
    6046: "2-4반_후303",
    6047: "체육관후면우_후210",
    6048: "체육관전면우_후210",
    6049: "시청각실_후509",
    6050: "시청각실_후509",
    6051: "시청각실_후509",
    6052: "2-7반_후306",
    6053: "2-6반_후305",
    6054: "식당전4_급101",
    6055: "2-1반_본304",
    6056: "식당전3_급101",
    6057: "2-1반_본304",
    6058: "식당전2_급101",
    6059: "전산실_본303",
    6060: "식당전1_급101",
    6061: "인쇄실_본302",
    6062: "식당중3_급101",
    6063: "스튜디오_본301-1",
    6064: "방송실_본301",
    6065: "5-8반_본310",
    6066: "5-7반_본309",
    6067: "2층도서실_후202",
    6068: "5-6반_본308",
    6069: "2층도서실_후202",
    6070: "5-5반_본307",
    6071: "6-8반_후406",
    6072: "2층도서실_후202",
    6073: "5-4반_본306",
    6074: "2층도서실_후202",
    6075: "5-3반_본305",
    6076: "6-7반_후405",
    6077: "연계형돌봄교실_후311",
    6078: "6-6반_후404",
    6079: "학부모실_후310",
    6080: "6-5반_후403",
    6081: "4협의실_후402",
    6082: "4-2반_후407",
    6083: "4-1반_후407-1",
    6084: "4-1반_후407-1",
    6085: "세척실_급103",
    6086: "조리실2_급105",
    6087: "조리실1_급105",
    6088: "영양사실_급106",
    6089: "휴게실_급109-1",
    6090: "전처리실_급109",
    6091: "식품창고_급107",
    6092: "위클래스_후201",
    6093: "4-3반_본401",
    6094: "4-4반_본402",
    6095: "4-5반_본403",
    6096: "5학년교사연구실_본410",
    6097: "남교사 휴게실_후503",
    6098: "2학년교사연구실_본311",
    6099: "여교사 휴게실_후502",
    6100: "위클래스_본211",
    6101: "6학년교사실_후508",
    6102: "1학년교사연구실_본109",
    6103: "6-4반_후507",
    6104: "6-3반_후506",
    6105: "식당중2_급101",
    6106: "식당중1_급101",
    6107: "돌봄준비실_후301-1",
    6108: "돌봄교실1_후301",
    6109: "돌봄교실1_후301",
    6110: "3층도서실덕트",
    6111: "4-6반_본404",
    6112: "3층도서실",
    6113: "3층도서실",
    6114: "6-2반_후505",
    6115: "6-1반_후504",
    6116: "아라반_후103",
    6117: "식당후면2_급101"
}

In [41]:
device_dict = {
    6007: "1-6반_본108",
    6008: "1-5반_본107",
    6009: "1-4반_본106",
    6010: "1-3반_본105",
    6028: "1-7반_본210",
    6029: "1-8반_본209",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="1학년", fig=False, W=None, H=600)

In [42]:
device_dict = {
    6044: "2-2반_후307-1",
    6045: "2-5반_후304",
    6046: "2-4반_후303",
    6052: "2-7반_후306",
    6053: "2-6반_후305",
    6055: "2-1반_본304",
    6057: "2-1반_본304",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="2학년", fig=False, W=None, H=600)

In [46]:
device_dict = {
    6001: "4-7반_본405",
    6082: "4-2반_후407",
    6083: "4-1반_후407-1",
    6084: "4-1반_후407-1",
    6093: "4-3반_본401",
    6094: "4-4반_본402",
    6095: "4-5반_본403",
    6111: "4-6반_본404",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="4학년", fig=False, W=None, H=600)

In [51]:
device_dict = {
    6002: "유치원복도1",
    6004: "누리반_후107",
    6005: "한울반_후106",
    6006: "가람반_후105",
    6116: "아라반_후103",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="유치반", fig=False, W=None, H=600)



In [62]:
device_dict = {
    6041: "돌봄협의실_후302",
    6042: "돌봄자료실_후307",
    6043: "방과후코디실_후307-1",
    6077: "연계형돌봄교실_후311",
    6107: "돌봄준비실_후301-1",
    6108: "돌봄교실1_후301",
    6109: "돌봄교실1_후301",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="돌봄교실", fig=False, W=None, H=600)



In [65]:
device_dict = {
    6003: "유치원교사실_후108",
    6023: "3학년연구실_후203",
    6096: "5학년교사연구실_본410",
    6098: "2학년교사연구실_본311",
    6101: "6학년교사실_후508",
    6102: "1학년교사연구실_본109",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="교사연구실", fig=False, W=None, H=600)

In [69]:
device_dict = {
    6079: "학부모실_후310",
    6081: "4협의실_후402",
    6089: "휴게실_급109-1",
    6092: "위클래스_후201",
    6097: "남교사 휴게실_후503",
    6099: "여교사 휴게실_후502",
    6100: "위클래스_본211",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="휴게실 등", fig=False, W=None, H=600)

In [78]:
# 우선 일주일치만 가지고 고찰
#df = df_buildID
#start_date, end_date = pd.to_datetime('2024-01-21 00:00:00'), pd.to_datetime('2024-01-28 00:00:00')  # 2024년 1월 21일(일)~27일(토)
start_date, end_date = pd.to_datetime('2024-01-22 00:00:00'), pd.to_datetime('2024-01-24 00:00:00')  # 2024년 1월 22일(월)~23일(화)
df = df_buildID[(df_buildID['colec_dt'] >= start_date) & (df_buildID['colec_dt'] <= end_date)]

In [79]:
device_dict = {
    6011: "체육관후면좌_후210",
    6012: "체육관전면좌_후210",
    6047: "체육관후면우_후210",
    6048: "체육관전면우_후210",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="체육관", fig=True, W=None, H=600)

In [ ]:
device_dict = {
    6011: "체육관후면좌_후210",
    6012: "체육관전면좌_후210",
    6013: "음악실1_본501-2",
    6014: "음악실1_본501-2",
    6016: "음악준비실_본501-4",
    6018: "개인연습실2_본501-5",
    6020: "개인연습실1_본501-6",
    6022: "무용실_본501",
    6024: "무용실_본501",
    6047: "체육관후면우_후210",
    6048: "체육관전면우_후210",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="예체능실", fig=True, W=None, H=600)

In [ ]:
device_dict = {
    6011: "체육관후면좌_후210",
    6012: "체육관전면좌_후210",
    6013: "음악실1_본501-2",
    6014: "음악실1_본501-2",
    6016: "음악준비실_본501-4",
    6018: "개인연습실2_본501-5",
    6020: "개인연습실1_본501-6",
    6022: "무용실_본501",
    6024: "무용실_본501",
    6025: "컴준비실_후208",
    6026: "컴퓨터실_후208-1",
    6027: "컴퓨터실_후208-1",
    6047: "체육관후면우_후210",
    6048: "체육관전면우_후210",
    6049: "시청각실_후509",
    6050: "시청각실_후509",
    6051: "시청각실_후509",
}

#tag_dict = {100003: '현재온도', 100004: '희망온도'}
tag_dict = {100003: '현재온도'}

# 모든 장치에 대해 데이터를 수집
all_tags_data = []
all_tags_names = []

for devID, device_name in device_dict.items():
    # 각 장치에 대해 데이터 선택
    tags = {key: select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
    
    # 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
    tag_data = [tags[key] for key in tag_dict.keys()]
    tag_names = [f"{device_name}, {tag_dict[key]}" for key in tag_dict.keys()]
    
    all_tags_data.extend(tag_data)
    all_tags_names.extend(tag_names)

# 모든 데이터를 하나의 그래프에 시각화
plot_devID_tagCD(all_tags_data, all_tags_names, device_name="예체능실", fig=True, W=None, H=600)

In [ ]:
device_dict = {

    6011: "체육관후면좌_후210",
    6012: "체육관전면좌_후210",
    6013: "음악실1_본501-2",
    6014: "음악실1_본501-2",

    6016: "음악준비실_본501-4",

    6018: "개인연습실2_본501-5",

    6020: "개인연습실1_본501-6",

    6022: "무용실_본501",
    6024: "무용실_본501",
    6025: "컴준비실_후208",
    6026: "컴퓨터실_후208-1",
    6027: "컴퓨터실_후208-1",

    6033: "행정실2_본203",
    6034: "행정실1_본203",
    6035: "탕비실_본204-2",
    6036: "교무실2_본202",
    6037: "교무실1_본202",
    6038: "교장실_본201",
    6039: "학교운영위원회실_본212",

    6047: "체육관후면우_후210",
    6048: "체육관전면우_후210",
    6049: "시청각실_후509",
    6050: "시청각실_후509",
    6051: "시청각실_후509",

    6054: "식당전4_급101",
    6056: "식당전3_급101",
    6058: "식당전2_급101",
    6059: "전산실_본303",
    6060: "식당전1_급101",
    6061: "인쇄실_본302",
    6062: "식당중3_급101",
    6063: "스튜디오_본301-1",
    6064: "방송실_본301",

    6067: "2층도서실_후202",
    6069: "2층도서실_후202",
    6072: "2층도서실_후202",
    6074: "2층도서실_후202",
    6079: "학부모실_후310",
    6081: "4협의실_후402",

    6085: "세척실_급103",
    6086: "조리실2_급105",
    6087: "조리실1_급105",
    6088: "영양사실_급106",
    6089: "휴게실_급109-1",
    6090: "전처리실_급109",
    6091: "식품창고_급107",
    6092: "위클래스_후201",

    6097: "남교사 휴게실_후503",
    6099: "여교사 휴게실_후502",
    6100: "위클래스_본211",

    6105: "식당중2_급101",
    6106: "식당중1_급101",
    6110: "3층도서실덕트",
    6112: "3층도서실",
    6113: "3층도서실",

    6117: "식당후면2_급101"
}